# Day 05.10.2021 Simulation

In [1]:
import pandas as pd
import numpy as np
from data_tools.simulation_data import SimulationDataCreator
from data_tools.simulation_run import RunSimulation
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", 40)
pd.set_option("display.precision", 5)

In [2]:
# upload, clean and prepare the data
simulation_data_creator = SimulationDataCreator("/Users/tom/dev/bbp-research/configuration_files/simulation_data_creator_05_10.json")
# upload the data
data_dict = simulation_data_creator.upload_data(["market_data", "market_data_1", "view_market_data"])
# convert invoker column
data_dict = simulation_data_creator.convert_invokers(data_dict, ["market_data", "market_data_1", "view_market_data"])
# merge the data
merged_data = simulation_data_creator.merge_markets(data_dict["market_data"]["data"], data_dict["market_data_1"]["data"], "timestamp", True)
merged_data = simulation_data_creator.merge_markets(merged_data, data_dict["view_market_data"]["data"], "timestamp", True)
# # add time
merged_data = simulation_data_creator.add_time(merged_data, "timestamp")
# data_dict = simulation_data_creator.add_times(data_dict, ["market_data", "market_data_1", "view_market_data"])
# cut hours in between 10:00 to 19:00
merged_data = simulation_data_creator.keep_data_between_hours(merged_data, 9, 19, "hour")

the value counts of column invoker is:

FCI_bid_0      488827
FCI_offer_0    487353
IBI_offer_0    230862
IBI_bid_0      230862
TA1_offer_0    182397
TA1_bid_0      182397
EBS_offer_0     78644
EBS_bid_0       78279
dtype: int64
1153855, 1153855
the value counts of column invoker is:

CITIQ_BARAK_offer_0       39726
CITIQ_BARAK_bid_0         39618
CREDITQ_BARAK_offer_0     26229
CREDITQ_BARAK_bid_0       26226
GOLDMANQ_BARAK_bid_0      12849
GOLDMANQ_BARAK_offer_0    12813
UBSQ_BARAK_offer_0         3964
UBSQ_BARAK_bid_0           3956
dtype: int64
164517, 164517
the value counts of column invoker is:

TRADAIR_LP12_bid_0      169202
TRADAIR_LP12_offer_0    169202
TRADAIR_LP6_bid_0       117315
TRADAIR_LP6_offer_0     117315
TRADAIR_LP15_bid_0      102903
TRADAIR_LP15_offer_0    102903
TRADAIR_LP1_bid_0        96859
TRADAIR_LP1_offer_0      96859
TRADAIR_LP13_offer_0     85380
TRADAIR_LP13_bid_0       85380
TRADAIR_LP5_bid_0        84825
TRADAIR_LP5_offer_0      84825
TRADAIR_LP3_offer_

## arena 1 simulation 1
- entering position condition: spread_tag_inc(TA1_offer, IBI_bid) < 0
- exiting position condition: current_best_bid >= best_offer_at_signal + 0.00005 or current_best_bid < best_bid_at_signal - 0.00005
- hit_brokers = ['FCI', 'IBI', 'TA1', 'IDB']
- view_brokers = none
- positon_fee = 0.00001
- max time in position = false

In [4]:
arena_1_sim_1 = RunSimulation(simulation_name = "arena 1 simulation 1", 
                              hit_brokers = ['FCI', 'IBI', 'TA1'],
                              view_brokers = [],
                              enter_conditions = "spread_tag_inc(TA1_offer, IBI_bid) < 0",
                              exit_conditions = "current_best_bid >= best_offer_at_signal + 0.00005 or current_best_bid < best_bid_at_signal - 0.00005",
                              positon_fee = 0.00001, 
                              max_time_in_position = False)
# initialize summary data frame                   
arena_1_sim_1.initialize_summary_df("increase")
# prepaer data for simulation
arena_1_sim_1_df = merged_data.copy()
sim_df_col_list, sim_df_np, sim_number_of_rows = arena_1_sim_1.prepaer_data_for_simulation(arena_1_sim_1_df)

In [7]:
arena_1_sim_1.initialize_summary_df("increase")
for i in range(sim_number_of_rows):
    # current market row
    current_market_data = sim_df_np[i]
    # check if entering condition holds
    if current_market_data[sim_df_col_list.index("IBI_bid_0")]:
        spread_tag_inc = current_market_data[sim_df_col_list.index("TA1_offer_0_rate")] - current_market_data[sim_df_col_list.index("IBI_bid_0_rate")]
        if spread_tag_inc < 0.0:
            best_offer_value, best_offer_broker, best_bid_value, best_bid_broker = arena_1_sim_1.find_bests_for_hit_brokers(current_market_data)
            temp_index = i + 1
            upper_limit_value = best_offer_value + 0.00005
            lower_limit_value = best_bid_value - 0.00005
            while temp_index < sim_number_of_rows and sim_df_np[temp_index][sim_df_col_list.index("best_bid_hit_rate")] < upper_limit_value and sim_df_np[temp_index][sim_df_col_list.index("best_bid_hit_rate")] > lower_limit_value:
                temp_index += 1
            if temp_index >= sim_number_of_rows:
                break
            else:
                arena_1_sim_1.documnet_position(current_market_data, sim_df_np[temp_index], spread_tag_inc)

In [6]:
df_to_present = arena_1_sim_1.create_simulation_summary()
df_to_present
print(f"")
print(df_to_present.shape[0])
print((df_to_present["timestamp_at_exiting"] - df_to_present["timestamp_at_entering"]).describe())
print(df_to_present["position_profit"].sum())

print((df_to_present['position_profit'] > 0).sum()/ df_to_present.shape[0])


1575
count      1575.00000
mean      24185.84952
std       29043.00590
min           1.00000
25%        4052.00000
50%       11720.00000
75%       34413.50000
max      208476.00000
dtype: float64
0.040810000000000006
0.5314285714285715


In [8]:
df_to_present = arena_1_sim_1.create_simulation_summary()
df_to_present
print(f"")
print(df_to_present.shape[0])
print((df_to_present["timestamp_at_exiting"] - df_to_present["timestamp_at_entering"]).describe())
print(df_to_present["position_profit"].sum())
print((df_to_present['position_profit'] > 0).sum()/ df_to_present.shape[0])


1575
count      1575.00000
mean      13692.01143
std       22349.99638
min           1.00000
25%         254.00000
50%        4035.00000
75%       14018.00000
max      158944.00000
dtype: float64
0.037570000000000006
0.6952380952380952


In [ ]:
# clean bad deals - insure no -1,0, what so ever deals
df_to_present["position_profit"].describe()

In [ ]:
# +-0.00005 to limits
df_to_present["position_profit"].sum()

In [ ]:
df_to_present["diff"] = df_to_present["timestamp_at_exiting"] - df_to_present["timestamp_at_entering"]
df_to_present["diff"].describe()

## arena 2 simulation 1
- entering position condition: spread_tag_inc(TA1_offer, IBI_bid) < 0
- exiting position condition: current_best_bid >= best_offer_at_signal + 0.00005 or current_best_bid < best_bid_at_signal - 0.00005
- hit_brokers = ["CITIQ_BARAK", "CREDITQ_BARAK", "UBSQ_BARAK", "GOLDMANQ_BARAK", "JPQ_BARAK"]
- view_brokers = [TA1, IBI]
- positon_fee = 0.00001
- max time in position = false

In [10]:
arena_2_sim_1 = RunSimulation(simulation_name = "arena 2 simulation 1", 
                              hit_brokers = ["CITIQ_BARAK", "CREDITQ_BARAK", "UBSQ_BARAK", "GOLDMANQ_BARAK"],
                              view_brokers = [],
                              enter_conditions = "spread_tag_inc(TA1_offer, IBI_bid) < 0",
                              exit_conditions = "current_best_bid >= best_offer_at_signal + 0.00005 or current_best_bid < best_bid_at_signal - 0.00005",
                              positon_fee = 0.00001, 
                              max_time_in_position = False)
# initialize summary data frame                   
arena_2_sim_1.initialize_summary_df("increase")
# prepaer data for simulation
arena_2_sim_1_df = merged_data.copy()
sim_df_col_list2, sim_df_np2, sim_number_of_rows2 = arena_2_sim_1.prepaer_data_for_simulation(arena_2_sim_1_df)

In [ ]:
arena_2_sim_1.initialize_summary_df("increase")
for i in range(sim_number_of_rows2):
    # current market row
    current_market_data = sim_df_np2[i]
    # check if entering condition holds
    if current_market_data[sim_df_col_list2.index("IBI_bid_0")]:
        spread_tag_inc = current_market_data[sim_df_col_list2.index("TA1_offer_0_rate")] - current_market_data[sim_df_col_list2.index("IBI_bid_0_rate")]
        if spread_tag_inc < 0.0:
            best_offer_value, best_offer_broker, best_bid_value, best_bid_broker = arena_2_sim_1.find_bests_for_hit_brokers(current_market_data)
            temp_index = i + 1
            upper_limit_value = best_offer_value + 0.0003
            lower_limit_value = best_bid_value - 0.0001
            # while temp_index < sim_number_of_rows and (sim_df_np[temp_index][sim_df_col_list.index("best_bid_hit_rate")] >= upper_limit_value or sim_df_np[temp_index][sim_df_col_list.index("best_bid_hit_rate")] < lower_limit_value):
            while temp_index < sim_number_of_rows2 and sim_df_np2[temp_index][sim_df_col_list2.index("best_bid_hit_rate")] < upper_limit_value and sim_df_np2[temp_index][sim_df_col_list2.index("best_bid_hit_rate")] > lower_limit_value:
                temp_index += 1
            if temp_index >= sim_number_of_rows2:
                break
            else:
                arena_2_sim_1.documnet_position(current_market_data, sim_df_np2[temp_index], spread_tag_inc)

In [ ]:
# limits + 0.0003 -0.0001
df_to_present = arena_2_sim_1.create_simulation_summary()
# df_to_presen

print(df_to_present.shape[0])
print((df_to_present["timestamp_at_exiting"] - df_to_present["timestamp_at_entering"]).describe())
print(df_to_present.shape[0])
print(df_to_present["position_profit"].sum())

In [ ]:
# limits + 0.0002 -0.00005
df_to_present = arena_2_sim_1.create_simulation_summary()
# df_to_presen

print(df_to_present.shape[0])
print((df_to_present["timestamp_at_exiting"] - df_to_present["timestamp_at_entering"]).describe())
print(df_to_present.shape[0])
print(df_to_present["position_profit"].sum())

In [ ]:
# limits + 0.0001 -0.00005
df_to_present = arena_2_sim_1.create_simulation_summary()
# df_to_presen

print(df_to_present.shape[0])
print((df_to_present["timestamp_at_exiting"] - df_to_present["timestamp_at_entering"]).describe())
print(df_to_present.shape[0])
print(df_to_present["position_profit"].sum())

In [ ]:
df_to_present["diff"] = df_to_present["timestamp_at_exiting"] - df_to_present["timestamp_at_entering"]
df_to_present["diff"].describe()

In [ ]:
df_to_present["position_profit"].sum()

## arena 3 simulation 1
- entering position condition: spread_tag_inc(TA1_offer, IBI_bid) < 0
- exiting position condition: current_best_bid >= best_offer_at_signal + 0.00005 or current_best_bid < best_bid_at_signal - 0.00005
- hit_brokers = ["FCI", "IBI", "TA1", "IDB", "CITIQ_BARAK", "CREDITQ_BARAK","UBSQ_BARAK", "GOLDMANQ_BARAK", "JPQ_BARAK"]
- view_brokers = None
- positon_fee = 0.00001
- max time in position = false

In [ ]:
arena_3_sim_1 = RunSimulation(simulation_name = "arena 3 simulation 1", 
                              hit_brokers = ["FCI", "IBI", "TA1", "IDB", "CITIQ_BARAK", "CREDITQ_BARAK", "UBSQ_BARAK", "GOLDMANQ_BARAK", "JPQ_BARAK"],
                              view_brokers = [],
                              enter_conditions = "spread_tag_inc(TA1_offer, IBI_bid) < 0",
                              exit_conditions = "current_best_bid >= best_offer_at_signal + 0.00005 or current_best_bid < best_bid_at_signal - 0.00005",
                              positon_fee = 0.00001, 
                              max_time_in_position = False)
# initialize summary data frame                   
arena_3_sim_1.initialize_summary_df("increase")
# prepaer data for simulation
arena_3_sim_1_df = merged_data.copy()
sim_df_col_list3, sim_df_np3, sim_number_of_rows3 = arena_3_sim_1.prepaer_data_for_simulation(arena_3_sim_1_df)

In [ ]:
for i in range(sim_number_of_rows3):
    # current market row
    current_market_data = sim_df_np3[i]
    # check if entering condition holds
    if current_market_data[sim_df_col_list3.index("IBI_bid_0")]:
        spread_tag_inc = current_market_data[sim_df_col_list3.index("TA1_offer_0_rate")] - current_market_data[sim_df_col_list3.index("IBI_bid_0_rate")]
        if spread_tag_inc < 0.0:
            best_offer_value, best_offer_broker, best_bid_value, best_bid_broker = arena_3_sim_1.find_bests_for_hit_brokers(current_market_data)
            temp_index = i + 1
            upper_limit_value = best_offer_value + 0.0001
            lower_limit_value = best_bid_value - 0.0001
            # while temp_index < sim_number_of_rows and (sim_df_np[temp_index][sim_df_col_list.index("best_bid_hit_rate")] >= upper_limit_value or sim_df_np[temp_index][sim_df_col_list.index("best_bid_hit_rate")] < lower_limit_value):
            while temp_index < sim_number_of_rows3 and sim_df_np3[temp_index][sim_df_col_list3.index("best_bid_hit_rate")] < upper_limit_value and sim_df_np3[temp_index][sim_df_col_list3.index("best_bid_hit_rate")] > lower_limit_value:
                temp_index += 1
            if temp_index >= sim_number_of_rows3:
                break
            else:
                arena_3_sim_1.documnet_position(current_market_data, sim_df_np3[temp_index], spread_tag_inc)

In [ ]:
df_to_present = arena_3_sim_1.create_simulation_summary()
df_to_present

In [ ]:
df_to_present["diff"] = df_to_present["timestamp_at_exiting"] - df_to_present["timestamp_at_entering"]
df_to_present["diff"].describe()

In [ ]:
df_to_present["position_profit"].sum()

In [ ]:
df_to_present.shape[0]

## arena 1 simulation 2
- entering position condition: spread_tag_inc(TA1_offer, IBI_bid) < 0
- exiting position condition: current_best_bid >= best_offer_at_signal + 0.00005 or current_best_bid < best_bid_at_signal - 0.00005 AND 3 horese in the offer direction
- hit_brokers = ['FCI', 'IBI', 'TA1', 'IDB']
- view_brokers = LP's
- positon_fee = 0.00001
- max time in position = false

## arena 2 simulation 2
- entering position condition: spread_tag_inc(TA1_offer, IBI_bid) < 0
- exiting position condition: current_best_bid >= best_offer_at_signal + 0.00005 or current_best_bid < best_bid_at_signal - 0.00005 AND 3 horese in the offer direction
- hit_brokers = ["CITIQ_BARAK", "CREDITQ_BARAK", "UBSQ_BARAK", "GOLDMANQ_BARAK", "JPQ_BARAK"]
- view_brokers = [TA1, IBI] + LP's
- positon_fee = 0.00001
- max time in position = false